<a href="https://colab.research.google.com/github/gshaffer22/GIS3_Adv_Python/blob/main/CCVI_Polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Permissions
Run these cells for the initial imports. The arcgis package is used in this analysis. Once installed, the session may restart, so it is important to do this early on.

In [1]:

# analysis and mapping
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
#import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from pprint import pprint

# file management stuff
import datetime
import glob
import urllib
import zipfile
from zipfile import ZipFile

# earth engine
import ee
import geemap.foliumap as geemap
#from geemap import geojson_to_ee, ee_to_geojson




In [2]:
#Install arcgis if not already installed
!pip install arcgis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.5/738.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.6 MB/s eta 0:00:00
  Created wheel for pylerc: filename=pylerc-4.0-py3-none-any.whl size=738090 sha256=23eda8dc57ff2dec545ac2e3514ec38b1a8e4985e73c0327d3d8d350cb39c42a
  Stored in directory: /root/.cache/pip/wheels/fa/44/c1/dee493b5ef79483d0276093205256ae13c97009c847e54c580
Successfully built pylerc
  Attempting uninstall: numpy
    Found exist

In [2]:
# then import GIS to be able to read and access ArcGIS online pages and layers
from arcgis.gis import GIS

gis = GIS()   # anonymous


# Authenticate your session
Run these chunks to authenticate your session and make sure you change the ee.Initialize(project= "YOUR PROJECT NAME) to your own earth engine project name.

In [3]:
ee.Authenticate()

In [4]:
# start the ee session
# Initialize the library.
ee.Initialize(project='useful-builder-474619-f1') # replace with your own earth engine project name


# Input Data & Initial Selection
Input the desired URL and county for which you wish to complete this analysis for. The URL should be:
*   data in the United States
*   a feature layer with polygons
*   open access from ArcGIS online

Example datasets include:

*   California Parcel Data https://www.maps.arcgis.com/home/item.html?id=f937000d00c340fb8b502fdd16e30882
*   Indigenous Land Data https://www.maps.arcgis.com/home/item.html?id=e46f229101f3438fbe123374e14f98f4







In [5]:
#INPUTS:
# Input the url to the arcgis data you wish to use, ensure the data is a feature layer with polygons AND it is in the United States
input_url = "https://www..com/home/item.html?id=e49e181ac82c46edac3ae601ebb3ef2d"

# this analysis focuses on data in the United States by county so please select a county name with the first letter of each word capitalized and spelled correctly

# Input the full name (first letter of each word capitalized) of the county you wish to query
countyname = "Mariposa County"

In [6]:
# correctly formatting your desired input data:
# for this analysis, your data should be from ArcGIS Online and a feature layer with polygon data
# examples include:

# Split on 'id=' and take the last part
# if statement to make sure the link is correct
if "id" in input_url:
    item_id = input_url.split("id=")[-1]
    print(item_id)
else:
    print("The link you chose is not compatible for this analysis. See the example links and make sure they align")


e49e181ac82c46edac3ae601ebb3ef2d


In [48]:
# This is where the input layer is retrieved for the analysis
# the try-except helps with error checking, in case the link you submitted was not valid
try:
  item = gis.content.get(item_id) # load your data, using the numbers/letters after the id= in the url at the top of the webpage

  print(item)
except:
  print("The arcgis online link you submitted is not compatible with this analysis. Please make sure the url contains an ID and that the data is in the US, a feature layer, is polygon data, and is open access")



<Item title:"USA Parks" type:Feature Layer Collection owner:esri_dm>


In [8]:
# getting the layer from the input data from arcgis online
try:
  data_layer = item.layers[0]

  print(data_layer)
except:
  print("The url you provided does not have a valid layer that can be used for this analysis.")

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Detailed_Parks/FeatureServer/0">


# Creating dataframes for input data
This section creates a spatial dataframe and a geodataframe from the data with the correct coordinate reference system.

In [9]:
# creating a spatial dataframe from your input data
df = data_layer.query().sdf

In [10]:
# converting the sdf to a gdf for your input data
gdf = gpd.GeoDataFrame(df, geometry='SHAPE')  # 'SHAPE' is the default geometry column in ArcGIS


In [11]:
# checking to make sure the data came in correctly
gdf.tail()

,OBJECTID,NAME,FEATTYPE,MNFC,SQMI,Shape__Area,Shape__Length,SHAPE
61152,45497,National Forest Huron-Manistee,Regional park,7170,1593.88,0.464994,61.939728,"MULTIPOLYGON (((-85.53495 42.48832, -85.53484 ..."
61153,49295,Thunder Bay National Marine Sanctuary,National park or forest,7170,4265.79,1.264766,7.178735,"POLYGON ((-82.32952 44.51284, -82.34087 44.512..."
61154,54840,Edwin B Forsythe National Wildlife Refuge,National park or forest,7170,96.54,0.026238,8.377524,"MULTIPOLYGON (((-74.40548 39.4415, -74.40494 3..."
61155,58018,Adirondack Park,State park or forest,7170,8966.54,2.603443,9.26796,"POLYGON ((-74.51001 43.05377, -74.51006 43.053..."
61156,59958,White Mountain National Forest,National park or forest,7170,1255.88,0.36584,7.748487,"MULTIPOLYGON (((-71.78149 43.79145, -71.78234 ..."


In [12]:
# checking that all the records came in
print(len(gdf))

61157


In [13]:
# check for the coordinate system of your dataset
print(gdf.crs)

None


In [14]:
# setting a coordinate system so this can be converted to WGS 84 (WGS is standard for global data), unless there is already a crs set
if gdf.crs == None:
  gdf.set_crs("EPSG:4326", inplace=True)
else:
  gdf.set_crs(gdf.crs)

# Getting and processing the county boundary data
The state boundary data is also retrieved from an open source data layer on ArcGIS online. This is processed in this section.

This is the link to the county boundary data: https://www.arcgis.com/home/item.html?id=36a7fa832aea4b7e8266a8d983afa4ac

In [15]:
# this is where the county boundaries layer is retrieved
county = gis.content.get("36a7fa832aea4b7e8266a8d983afa4ac") # load your data, using the numbers/letters after the id= in the url at the top of the webpage
print(county)

<Item title:"United States County Boundaries" type:Feature Layer Collection owner:esri_dm>


In [16]:
# getting the layer from the county boundary data from arcgis online
county_layer = county.layers[2]
print(county_layer)

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Boundaries_2023/FeatureServer/2">


In [17]:
# creating a spatial dataframe from the county data
county_df = county_layer.query().sdf

In [18]:
# making a geodataframe from the state data
county_gdf = gpd.GeoDataFrame(county_df, geometry='SHAPE')  # 'SHAPE' is the default geometry column in ArcGIS


In [19]:
# printing the last 5 entries in the state dataframe to make sure they are converted to the gdf correctly
county_gdf.tail()

,OBJECTID,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,POPULATION_2020,POP20_SQMI,Shape__Area,Shape__Length,SHAPE
3138,2803,Utah County,Utah,UT,49,049,49049,718194,334.94,2144.22,659399,307.5,0.586729,5.106125,"POLYGON ((-111.59389 40.57707, -111.59379 40.5..."
3139,2802,Uintah County,Utah,UT,49,047,49047,36706,8.15,4504.05,35620,7.9,1.232563,5.395294,"POLYGON ((-109.04896 40.66261, -109.04895 40.6..."
3140,3055,Burnett County,Wisconsin,WI,55,013,55013,16828,19.12,880.22,16526,18.8,0.264132,2.784881,"POLYGON ((-92.04964 46.15761, -92.04967 46.155..."
3141,3054,Buffalo County,Wisconsin,WI,55,011,55011,13234,18.65,709.59,13317,18.8,0.207539,2.265479,"POLYGON ((-92.00315 44.59684, -92.00093 44.596..."
3142,3053,Brown County,Wisconsin,WI,55,009,55009,274271,516.28,531.24,268740,505.9,0.155565,3.253008,"MULTIPOLYGON (((-87.9809 44.53818, -87.98061 4..."


In [20]:
print(county_gdf.crs)

None


In [21]:
# the state data did not have any coordinate reference system attatched
# EPSG:4326 = WGS 84
# this is the same as the initial dataset
county_gdf.set_crs("EPSG:4326", inplace=True)


,OBJECTID,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,POPULATION_2020,POP20_SQMI,Shape__Area,Shape__Length,SHAPE
0,702,Benton County,Indiana,IN,18,007,18007,8644,21.26,406.51,8719,21.4,0.112026,1.383307,"POLYGON ((-87.50164 40.73697, -87.49919 40.736..."
1,703,Blackford County,Indiana,IN,18,009,18009,11966,72.27,165.58,12112,73.1,0.045543,0.866112,"POLYGON ((-85.31627 40.56734, -85.30245 40.567..."
2,704,Boone County,Indiana,IN,18,011,18011,75892,179.32,423.22,70812,167.3,0.115691,1.419801,"POLYGON ((-86.24074 39.92607, -86.2411 39.9261..."
3,705,Brown County,Indiana,IN,18,013,18013,15610,49.3,316.63,15475,48.9,0.085508,1.182458,"POLYGON ((-86.08561 39.3442, -86.08443 39.3279..."
4,706,Carroll County,Indiana,IN,18,015,18015,20318,54.18,375.02,20306,54.1,0.103313,1.465989,"POLYGON ((-86.73692 40.73755, -86.72819 40.737..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,2803,Utah County,Utah,UT,49,049,49049,718194,334.94,2144.22,659399,307.5,0.586729,5.106125,"POLYGON ((-111.59389 40.57707, -111.59379 40.5..."
3139,2802,Uintah County,Utah,UT,49,047,49047,36706,8.15,4504.05,35620,7.9,1.232563,5.395294,"POLYGON ((-109.04896 40.66261, -109.04895 40.6..."
3140,3055,Burnett County,Wisconsin,WI,55,013,55013,16828,19.12,880.22,16526,18.8,0.264132,2.784881,"POLYGON ((-92.04964 46.15761, -92.04967 46.155..."
3141,3054,Buffalo County,Wisconsin,WI,55,011,55011,13234,18.65,709.59,13317,18.8,0.207539,2.265479,"POLYGON ((-92.00315 44.59684, -92.00093 44.596..."


# Joining your input data with the state boundaries
This section joins the two geodataframes (your input data AND the state boundary data).

In [56]:
# Making a joined dataframe with the state and input data to be able to query the data by the initial county you chose a the beginning
joined_gdf = gpd.sjoin(
    gdf,
    county_gdf,
    how='inner',
    predicate='intersects',
    lsuffix='input',
    rsuffix='county'
)


In [35]:
list(joined_gdf.columns)


['OBJECTID_input',
 'NAME_input',
 'FEATTYPE',
 'MNFC',
 'SQMI_input',
 'Shape__Area_input',
 'Shape__Length_input',
 'SHAPE',
 'index_county',
 'OBJECTID_county',
 'NAME_county',
 'STATE_NAME',
 'STATE_ABBR',
 'STATE_FIPS',
 'COUNTY_FIPS',
 'FIPS',
 'POPULATION',
 'POP_SQMI',
 'SQMI_county',
 'POPULATION_2020',
 'POP20_SQMI',
 'Shape__Area_county',
 'Shape__Length_county']

In [36]:
joined_gdf.tail()

,OBJECTID_input,NAME_input,FEATTYPE,MNFC,SQMI_input,Shape__Area_input,Shape__Length_input,SHAPE,index_county,OBJECTID_county,...,STATE_FIPS,COUNTY_FIPS,FIPS,POPULATION,POP_SQMI,SQMI_county,POPULATION_2020,POP20_SQMI,Shape__Area_county,Shape__Length_county
61155,58018,Adirondack Park,State park or forest,7170,8966.54,2.603443,9.26796,"POLYGON ((-74.51001 43.05377, -74.51006 43.053...",1383,1839,...,36,019,36019,79556,71.19,1117.51,79843,71.4,0.328882,2.539754
61156,59958,White Mountain National Forest,National park or forest,7170,1255.88,0.36584,7.748487,"MULTIPOLYGON (((-71.78149 43.79145, -71.78234 ...",1311,1767,...,33,003,33003,52707,53.05,993.49,50107,50.4,0.288129,2.950526
61156,59958,White Mountain National Forest,National park or forest,7170,1255.88,0.36584,7.748487,"MULTIPOLYGON (((-71.78149 43.79145, -71.78234 ...",1314,1770,...,33,009,33009,92087,52.63,1749.7,91118,52.1,0.508005,3.871571
61156,59958,White Mountain National Forest,National park or forest,7170,1255.88,0.36584,7.748487,"MULTIPOLYGON (((-71.78149 43.79145, -71.78234 ...",1408,1187,...,23,017,23017,58114,26.71,2175.76,57777,26.6,0.637656,5.45194
61156,59958,White Mountain National Forest,National park or forest,7170,1255.88,0.36584,7.748487,"MULTIPOLYGON (((-71.78149 43.79145, -71.78234 ...",1313,1769,...,33,007,33007,31180,17.04,1830.31,31268,17.1,0.53814,4.615224


In [31]:
county_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [37]:
# checking if the county name variable you input is valid
match = joined_gdf[joined_gdf['NAME_county'] == countyname]

if len(match) > 0:
    print(f"Your selection of {countyname} is a valid county name.")
else:
    print("Your county name is NOT valid.")

# IF your county name is not valid, input a new state name in the initial code block where it is defined (where your inputs are)

Your selection of Mariposa County is a valid county name.


In [38]:
# filtering the data and creating a new selection based on your filter
# returning how many, if any, features are located in your county
filtered_gdf = joined_gdf[joined_gdf['NAME_county'] == countyname]
# Check that any rows exist:

if filtered_gdf.empty:
    print("No features found for that county.")
else:
    print(f"Found {len(filtered_gdf)} features in {countyname}.")


Found 4 features in Mariposa County.


In [39]:
# setting the correct WGS 84 coordinate reference system
filtered_gdf = filtered_gdf.to_crs(epsg=4326)


In [40]:
# getting the centroid for the state you selected to have the correct map center
cent = filtered_gdf.centroid
centlong = cent.x.values[0]
centlat = cent.y.values[0]


/tmp/ipython-input-3434834619.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent = filtered_gdf.centroid


In [41]:
# making a feature collection layer that ee likes
ee_county = geemap.gdf_to_ee(filtered_gdf)


In [50]:
# creating a map to check that the features appear correctly
# create the map variable
m = folium.Figure(width=800, height=600)
Map = geemap.Map(zoom=7)
Map.add_to(m)

# add your ee layer with the correct styling so it appears
Map.addLayer(
    ee_county,
    {'color': 'red', 'fillColor': 'FF000080', 'width': 2},
    f"Features in {countyname}"
)

# zoom to the county bounds
minx, miny, maxx, maxy = filtered_gdf.total_bounds
Map.fit_bounds([[miny, minx], [maxy, maxx]])

# add layer control to be able to control the map
Map.add_layer_control()

m



In [51]:
# save this map as an html export
outhtml = f'{countyname}_{item.title}_Map.html'
m.save(outhtml)
print(f'Saved {outhtml}')


Saved Mariposa County_USA Parks_Map.html


# Getting the CCVI Data for analysis
The CCVI data is from arcgis online as well. It is linked here:
https://www.arcgis.com/home/item.html?id=0bd7d81a57224cbea9a33d7b1358bad4

CCVI is the [Climate Conflict Vulnerability Index](https://climate-conflict.org/www), commissioned by the German Federal Foreign Office to show the intersection of climate and conflict related impacts across ther world.

In [43]:
# getting CCVI data from arcgis online in the same method
ccvi_gis = gis.content.get("0bd7d81a57224cbea9a33d7b1358bad4") # load your data, using the numbers/letters after the id= in the url at the top of the webpage
ccvi_gis


<Item title:"Climate Conflict Vulnerability Index (CCVI)" type:Feature Layer Collection owner:esri_DE_content>

In [44]:
ccvi_layer = ccvi_gis.layers[0]
ccvi_layer

<FeatureLayer url:"https://services2.arcgis.com/jUpNdisbWqRpMo35/arcgis/rest/services/climate_conflict_points_layer/FeatureServer/0">

In [45]:
# creating a spatial dataframe for ccvi
ccvi_df = ccvi_layer.query().sdf

In [52]:
# converting the sdf to a gdf for your input data
ccvi_gdf = gpd.GeoDataFrame(ccvi_df, geometry='SHAPE')  # 'SHAPE' is the default geometry column in ArcGIS


In [53]:
# checking to make sure the data came in correctly
ccvi_gdf.tail()

,pgid,lat,lon,iso3,CLI_longterm_temperature_anomal,CLI_current_floods_raw,CLI_current_drought_raw,CLI_accumulated_cyclones_raw,CLI_current_cyclones_raw,CLI_accumulated_drought_raw,...,CON,VUL,CLI_risk,CON_risk,CCVI,EXP_pop_density,EXP_pop_density_raw,EXP_pop_count,ObjectId,SHAPE
60769,245975,80.75,47.25,Russische Föderation,2.309,68.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.034,0.008,0.027,0.004,0.027,5.243,60628,POINT (5259845.94 16038305.761)
60770,246003,80.75,61.25,Russische Föderation,2.241,79.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.037,0.009,0.03,0.005,0.035,13.427,60699,POINT (6818318.811 16038305.761)
60771,245255,80.25,47.25,Russische Föderation,2.213,72.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.032,0.008,0.025,0.004,0.024,4.204,60761,POINT (5259845.94 15700993.743)
60772,245256,80.25,47.75,Russische Föderation,2.225,61.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.032,0.008,0.026,0.004,0.025,5.394,60762,POINT (5315505.685 15700993.743)
60773,245257,80.25,48.25,Russische Föderation,2.238,80.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.032,0.008,0.025,0.004,0.024,9.003,60763,POINT (5371165.431 15700993.743)


In [54]:
# checking if the CCVI data already has a coordinate reference system
print(ccvi_gdf.crs)

None


In [55]:
# since there is no coordinate reference system, convert it to WGS 84
ccvi_gdf.set_crs("EPSG:4326", inplace=True)


,pgid,lat,lon,iso3,CLI_longterm_temperature_anomal,CLI_current_floods_raw,CLI_current_drought_raw,CLI_accumulated_cyclones_raw,CLI_current_cyclones_raw,CLI_accumulated_drought_raw,...,CON,VUL,CLI_risk,CON_risk,CCVI,EXP_pop_density,EXP_pop_density_raw,EXP_pop_count,ObjectId,SHAPE
0,66461,-43.75,-69.75,Argentinien,1.07,0.0,-0.633,0.0,0.0,-0.606,...,0.0,0.155,0.045,0.001,0.035,0.021,0.155,345.315,1,POINT (-7764534.48283 -5426835.26547)
1,66462,-43.75,-69.25,Argentinien,1.07,0.0,-0.32,0.0,0.0,-0.425,...,0.0,0.208,0.044,0.01,0.035,0.018,0.131,292.229,2,POINT (-7708874.73743 -5426835.26547)
2,66463,-43.75,-68.75,Argentinien,1.071,0.0,-0.394,0.0,0.0,-0.414,...,0.0,0.192,0.034,0.001,0.026,0.011,0.079,177.461,3,POINT (-7653214.99204 -5426835.26547)
3,66464,-43.75,-68.25,Argentinien,1.072,0.0,-0.545,0.0,0.0,-0.47,...,0.0,0.22,0.032,0.001,0.024,0.008,0.057,128.196,4,POINT (-7597555.24664 -5426835.26547)
4,66465,-43.75,-67.75,Argentinien,1.072,0.0,-0.524,0.0,0.0,-0.51,...,0.0,0.224,0.031,0.001,0.024,0.007,0.052,115.166,5,POINT (-7541895.50124 -5426835.26547)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60769,245975,80.75,47.25,Russische Föderation,2.309,68.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.034,0.008,0.027,0.004,0.027,5.243,60628,POINT (5259845.93998 16038305.76075)
60770,246003,80.75,61.25,Russische Föderation,2.241,79.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.037,0.009,0.03,0.005,0.035,13.427,60699,POINT (6818318.81109 16038305.76075)
60771,245255,80.25,47.25,Russische Föderation,2.213,72.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.032,0.008,0.025,0.004,0.024,4.204,60761,POINT (5259845.93998 15700993.74273)
60772,245256,80.25,47.75,Russische Föderation,2.225,61.0,<NA>,0.0,0.0,<NA>,...,0.0,0.192,0.032,0.008,0.026,0.004,0.025,5.394,60762,POINT (5315505.68538 15700993.74273)


In [59]:
ccvi_gdf = ccvi_gdf.to_crs(filtered_gdf.crs)
